# Sandbox notebook

In [1]:
import gc
import os
import sys
import math
import glob
import tqdm
import random
import numpy as np
from tqdm import tqdm
from time import sleep

In [2]:
import pandas as pd
import xml.etree.cElementTree as ET

In [3]:
import cv2
import dlib
from imutils import face_utils
from skimage.feature import hog
from skimage import data,exposure

## Aggregating OpenFace csv to xml

In [6]:
path_openface_data = '/media/amogh/Stuff/oface'
list_csv_files = glob.glob(path_openface_data + '/*.csv')

In [7]:
os.path.splitext(os.path.basename(list_csv_files[0]))[0]

'VIDEO_08-female-frustration_20180516_110946_1744913117'

In [10]:
df_sample = pd.read_csv(list_csv_files[-30])
df_sample

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.000,1.00,0,0.556916,0.314565,-0.768696,0.556912,0.314564,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0,0.034,1.00,0,0.556916,0.314565,-0.768696,0.556912,0.314564,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,0.067,1.00,0,0.556916,0.314565,-0.768696,0.556912,0.314564,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0,0.101,1.00,0,0.556916,0.314565,-0.768696,0.556912,0.314564,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,0.135,1.00,0,0.556916,0.314565,-0.768696,0.556912,0.314564,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,0,0.169,1.00,0,0.556916,0.314565,-0.768696,0.556912,0.314564,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,0,0.202,1.00,0,0.556916,0.314565,-0.768696,0.556912,0.314564,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,0,0.236,1.00,0,0.556916,0.314565,-0.768696,0.556912,0.314564,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,0,0.270,1.00,0,0.556916,0.314565,-0.768696,0.556912,0.314564,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,0,0.304,1.00,0,0.556916,0.314565,-0.768696,0.556912,0.314564,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
for index, row in df_sample.iterrows():
    a = (row[' x_0':' x_67'])
#     a = (row[' y_0':' y_67'])
    print list(a)
#     print row.apply(lambda x: x.str.contains('^x_'))
#     print a

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [ ]:
def getXMLFromOpenFaceData(path_folder_openface_data):
    """
    Generates XML data file from a folder containing the output csv files from OpenFace.
    """
    
    list_csv_files = glob.glob(path_folder_openface_data + '/*.csv')
    
    #Generate XML file.
    root = ET.Element("dataset")
    name = ET.SubElement(root, "name").text = "Labelled faces"
    comment = ET.SubElement(root, "comment").text = "These are labelled images from Bagamoyo"
    images = ET.SubElement(root, "images")

    # Read csv file and populate the XML data.
    for path_csv in tqdm(list_csv_files):
        name_video = os.path.splitext(os.path.basename(path_csv))[0]
        df_data = pd.read_csv(path_csv)
        
        # Iterate over each frame in this video's output
        for index, row in df_data.iterrows():
            #define the image node for which box will be a child.
            frame_num = row.frame
            image_node_file_name = name_video + " " + "%03d"%frame_num + ".jpg"
            #check if the tag for frame already exists, create if not.
            for image_elem in images.iter():
#                 print(image_elem.get('file'))
                if(image_elem.get('file') == image_node_file_name):
                      image_node = image_elem
                else:
                      image_node = ET.SubElement(images, "image", file=image_node_file_name)
            #get box coordinates
            list_x_coord_landmarks = list(row[' x_0':' x_67']) 
            list_y_coord_landmarks = list(row[' y_0':' y_67'])
            left = int(min(list_x_coord_landmarks))
            right = int(max(list_x_coord_landmarks))
            top = int(min(list_y_coord_landmarks))
            bottom = int(max(list_y_coord_landmarks))
            width = right - left
            height = bottom - top
            #add box node
            box = ET.SubElement(image_node, "box", height=str(height), left=str(left), top=str(top), width=str(width))

                      
getXMLFromOpenFaceData(path_openface_data)  

  0%|          | 0/166 [00:00<?, ?it/s]

In [34]:
def debug():
    root = ET.Element("dataset")
    name = ET.SubElement(root, "name", file='a')
#     name = ET.SubElement(root, "name", file='a')
    name = ET.SubElement(root, "name", file='a')
    print([a for a in root])
    tree = ET.ElementTree(root)
    tree.write('try.xml')
debug()

[<Element 'name' at 0x7f001761a720>, <Element 'name' at 0x7f001761a7e0>]
